In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pywt

# Parameters
ro0 = 1
ro1 = -1
n = 512
h = 2 * np.pi / n
a = 1
d = 10

# Pre-computations
I = 4 * (np.log(2) * np.sin(h / 2) + 2 * (np.sin(h / 2) * np.log(np.sin(h / 2)) - np.sin(h / 2)))
B = a * np.log(a) - a / 2 * np.log(2 * a**2)
A = 2 * h * B

# Initialize arrays
teta = np.zeros(n)
f = np.zeros(n)
k = np.zeros(n)
mat = np.zeros((n, n))

# Calculate teta, f, and k
for i in range(n):
    teta[i] = i * h
    f[i] = ro0 * (np.log(d) - 0.5 * np.log(a**2 + d**2 - 2 * a * d * np.cos(teta[i])))

    if i == 0:
        k[i] = A - a / 2 * I
    else:
        k[i] = B - a / 2 * (np.log(2 * np.sin(teta[i] / 2)))
        if i == 1 or i == n - 1:
            k[i] *= h / 3
        else:
            k[i] *= 2 * h / 3 * (1 + i % 2)

# Construct the matrix mat
for i in range(n):
    for j in range(n):
        mat[i, j] = k[(j - i + n) % n]
# Solve the system in the physical domain (MATLAB-like approach)
f_physical = -f
ros_physical = np.linalg.solve(mat, f_physical)

# Normalize and correct the solution (MATLAB-like approach)
su_physical = np.sum(ros_physical)
ros_physical = -su_physical / n + ros_physical + ro1 / (2 * np.pi * a)